In [1]:
from community_knapsack import *
from evaluation import evaluation
import random

In [17]:
# The algorithms to use for this evaluation:
exact_algorithms = [
    PBMultiAlgorithm.MEMOIZATION,
    PBMultiAlgorithm.ILP_SOLVER
]

approximation_algorithms = [
    PBMultiAlgorithm.GREEDY,
    PBMultiAlgorithm.RATIO_GREEDY,
    PBMultiAlgorithm.BRANCH_AND_BOUND,
    PBMultiAlgorithm.SIMULATED_ANNEALING,
    PBMultiAlgorithm.GENETIC_ALGORITHM
]

#### This notebook tests the scalability of the algorithms with an increasing number of dimensions and increasing project costs.
---

In [18]:
# The boundaries of this evaluation:

# We test between one and ten dimensions where each budget is randomly generated
# between 10000 and 100000 and the costs of each project are randomly generated
# from 10-20%, 10-30%, ..., 10-100%.

random.seed(181)

num_projects = 50
num_voters = 3000

start_dimensions = 1
end_dimensions = 5
step_dimensions = 1

min_cost = 0.1

start_max_cost = 20
end_max_cost = 100
step_max_cost = 20

budget = [random.randint(10_000, 100_000) for _ in range(end_dimensions//step_dimensions)]

In [19]:
x_axis = list(range(start_dimensions, end_dimensions + 1, step_dimensions))

In [20]:
y_axis = list(cost/100 for cost in range(start_max_cost, end_max_cost + 1, step_max_cost))

In [21]:
problems = evaluation.generate_multi_problems(
    num_project_bounds=[(num_projects, num_projects)],
    num_voters_bounds=[(num_voters, num_voters)],
    dimension_bounds=x_axis,
    budget_bounds=[[[(b, b) for b in budget[:d]]] for d in x_axis],
    cost_bounds=[[[(int(b * min_cost), int(b * curr_cost)) for bid, b in enumerate(budget[:d])] for curr_cost in y_axis] for d in x_axis]
)

In [24]:
# Obtain the exact results for this evaluation:
exact_results = evaluation.solve_problems(
    problem_list=problems,
    algorithms=exact_algorithms,
    timeout=60,
    max_fail=-1,
    file_name='ext-dimensions-costs.json',
    output=True
)
exact_results = evaluation.format_3d(exact_results, x_axis, y_axis)

In [27]:
# Obtain the exact results for this evaluation:
approximation_results = evaluation.solve_problems(
    problem_list=problems,
    algorithms=approximation_algorithms,
    timeout=60,
    max_fail=-1,
    file_name='apx-dimensions-costs.json',
    output=True
)
approximation_results = evaluation.format_3d(approximation_results, x_axis, y_axis)

In [ ]:
evaluation.plot_3d(
    x_axis=x_axis,
    y_axis=[y*100 for y in y_axis],
    z_axes=evaluation.get_z_axes(exact_results, exact_algorithms, 2),
    x_label='# of Dimensions',
    y_label='Maximum Cost Generated\n(% of Budget)',
    z_label='Overall Runtime (ms)',
    labels=evaluation.get_labels(exact_algorithms),
    colors=evaluation.get_colors(exact_algorithms),
    sizes=evaluation.get_3d_sizes(exact_algorithms),
)

In [36]:
evaluation.plot_3d(
    x_axis=x_axis,
    y_axis=[y*100 for y in y_axis],
    z_axes=evaluation.get_z_axes(approximation_results, approximation_algorithms, 2),
    x_label='# of Dimensions',
    y_label='Maximum Cost Generated\n(% of Budget)',
    z_label='Overall Runtime (ms)',
    labels=evaluation.get_labels(approximation_algorithms),
    colors=evaluation.get_colors(approximation_algorithms),
    sizes=evaluation.get_3d_sizes(approximation_algorithms),
)

In [38]:
evaluation.plot_3d(
    x_axis=x_axis,
    y_axis=y_axis,
    z_axes=evaluation.get_z_axes(exact_results, [PBSingleAlgorithm.ILP_SOLVER], 0) + evaluation.get_z_axes(approximation_results, approximation_algorithms, 0),
    x_label='# of Dimensions',
    y_label='Maximum Cost Generated\n(% of Budget)',
    z_label='Overall Value (Votes)',
    labels=['IPS'] + evaluation.get_labels(approximation_algorithms),
    colors=['black'] + evaluation.get_colors(approximation_algorithms),
    alphas=[0.05],
    sizes=[100] + evaluation.get_3d_sizes(approximation_algorithms),
    y_tick_limits=(3, 3),
)